In [1]:
# Imports

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import optuna

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler, RobustScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.inspection import permutation_importance
from category_encoders import TargetEncoder, OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from scipy.stats import uniform, randint

#two choices for lightgbm
import lightgbm as lgb
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [2]:
# Datas
df=pd.read_csv('cleaned_data.csv')
df

,MIS_Status,LoanNr_ChkDgt,State,Zip,UrbanRural,Bank,BankState,NAICS,NoEmp,NewExist,...,FranchiseCode,IsFranchise,Term,RevLineCr,LowDoc,DisbursementGross,ApprovalDate,ApprovalFY,GrAppv,SBA_Appv
0,1,1000014003,IN,47711,0,FIFTH THIRD BANK,OH,451120,4,2.0,...,1,1,84,0,1,60000.0,1997-02-28,1997,60000.0,48000.0
1,1,1000024006,IN,46526,0,1ST SOURCE BANK,IN,722410,2,2.0,...,1,1,60,0,1,40000.0,1997-02-28,1997,40000.0,32000.0
2,1,1000034009,IN,47401,0,GRANT COUNTY STATE BANK,IN,621210,7,1.0,...,1,1,180,0,0,287000.0,1997-02-28,1997,287000.0,215250.0
3,1,1000044001,OK,74012,0,1ST NATL BK & TR CO OF BROKEN,OK,0,2,1.0,...,1,1,60,0,1,35000.0,1997-02-28,1997,35000.0,28000.0
4,1,1000054004,FL,32801,0,FLORIDA BUS. DEVEL CORP,FL,0,14,1.0,...,1,1,240,0,0,229000.0,1997-02-28,1997,229000.0,229000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899159,1,9995573004,OH,43221,0,JPMORGAN CHASE BANK NATL ASSOC,IL,451120,6,1.0,...,1,1,60,0,0,70000.0,1997-02-27,1997,70000.0,56000.0
899160,1,9995603000,OH,43221,0,JPMORGAN CHASE BANK NATL ASSOC,IL,451130,6,1.0,...,1,1,60,0,0,85000.0,1997-02-27,1997,85000.0,42500.0
899161,1,9995613003,CA,93455,0,"RABOBANK, NATIONAL ASSOCIATION",CA,332321,26,1.0,...,1,1,108,0,0,300000.0,1997-02-27,1997,300000.0,225000.0
899162,0,9995973006,HI,96830,0,BANK OF HAWAII,HI,0,6,1.0,...,1,1,60,0,1,75000.0,1997-02-27,1997,75000.0,60000.0


In [4]:
#Encoding categorical values with LabelEncoder
features = ['State', 'UrbanRural', 'Bank', 'BankState', 'NoEmp', 'NewExist', 'FranchiseCode',
            'CreateJob', 'RetainedJob', 'IsFranchise', 'Term', 'RevLineCr', 'LowDoc',
            'DisbursementGross', 'GrAppv']
target = 'MIS_Status'

# Séparation des features et de la target
X = df[features]
y = df[target]

cat_features = ['State', 'UrbanRural', 'Bank', 'BankState', 'NewExist', 'FranchiseCode', 
                'IsFranchise', 'RevLineCr', 'LowDoc']

label_encoders = {}
for col in cat_features:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col].astype(str))  # Convertir en string pour éviter les erreurs
    label_encoders[col] = le


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, stratify=y)

#Standardisation
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#Logistic regression 
model = LogisticRegression(max_iter=100, solver='lbfgs')
model.fit(X_train, y_train)

#Predictions and values
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

#Result
print(f"Accuracy: {accuracy:.4f}")
print("Confusion Matrix:")
print(conf_matrix)

/tmp/ipykernel_174283/2078396854.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = le.fit_transform(X[col].astype(str))  # Convertir en string pour éviter les erreurs
/tmp/ipykernel_174283/2078396854.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = le.fit_transform(X[col].astype(str))  # Convertir en string pour éviter les erreurs
/tmp/ipykernel_174283/2078396854.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_ind

Accuracy: 0.8430
Confusion Matrix:
[[ 3625 12157]
 [ 1956 72179]]


In [7]:
# Encoder les variables catégorielles
cat_features = ['State', 'UrbanRural', 'Bank', 'BankState', 'NewExist', 'FranchiseCode', 
                'IsFranchise', 'RevLineCr', 'LowDoc']
label_encoders = {}
for col in cat_features:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col].astype(str))
    label_encoders[col] = le

# Split des données
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, stratify=y)

# Fonction d'optimisation
def objective(trial):
    # Hyperparamètres à optimiser
    C = trial.suggest_loguniform("C", 0.001, 10)
    solver = trial.suggest_categorical("solver", ["lbfgs", "liblinear", "saga"])
    
    # Modèle
    model = LogisticRegression(C=C, solver=solver, max_iter=200)
    
    # Standardisation + entraînement
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    
    return accuracy_score(y_test, y_pred)

# Lancement d'Optuna
study = optuna.create_study(direction="maximize",study_name="LogisticRegression Accuracy", storage="sqlite:///optuna_study.db")
study.optimize(objective, n_trials=10)  # Limité à 10 essais pour la vitesse

# Résultats
print("Meilleurs paramètres :", study.best_params)
print("Meilleure Accuracy :", study.best_value)

/tmp/ipykernel_174283/216659503.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = le.fit_transform(X[col].astype(str))
/tmp/ipykernel_174283/216659503.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = le.fit_transform(X[col].astype(str))
/tmp/ipykernel_174283/216659503.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/p

Meilleurs paramètres : {'C': 5.004577800250009, 'solver': 'lbfgs'}
Meilleure Accuracy : 0.8412091150727894


In [8]:
#Predictions and values
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

#Result
print(f"Accuracy: {accuracy:.4f}")
print("Confusion Matrix:")
print(conf_matrix)

Accuracy: 0.2140
Confusion Matrix:
[[12427  3355]
 [67323  6812]]


/home/utilisateur/Documents/accord_pret/loan_project/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


In [9]:
#Encoding categorical values with LabelEncoder
features = ['State', 'UrbanRural', 'Bank', 'BankState', 'NoEmp', 'NewExist', 'FranchiseCode',
            'CreateJob', 'RetainedJob', 'IsFranchise', 'Term', 'RevLineCr', 'LowDoc',
            'DisbursementGross', 'GrAppv']
target = 'MIS_Status'

# Séparation des features et de la target
X = df[features]
y = df[target]

cat_features = ['State', 'UrbanRural', 'Bank', 'BankState', 'NewExist', 'FranchiseCode', 
                'IsFranchise', 'RevLineCr', 'LowDoc']

label_encoders = {}
for col in cat_features:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col].astype(str))  # Convertir en string pour éviter les erreurs
    label_encoders[col] = le


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, stratify=y)

#Standardisation
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#Logistic regression 
model = LogisticRegression(max_iter=100, C=5.004577800250009, solver='lbfgs')
model.fit(X_train, y_train)

#Predictions and values
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

#Result
print(f"Accuracy: {accuracy:.4f}")
print("Confusion Matrix:")
print(conf_matrix)

/tmp/ipykernel_174283/3870844073.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = le.fit_transform(X[col].astype(str))  # Convertir en string pour éviter les erreurs
/tmp/ipykernel_174283/3870844073.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = le.fit_transform(X[col].astype(str))  # Convertir en string pour éviter les erreurs
/tmp/ipykernel_174283/3870844073.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_ind

Accuracy: 0.8430
Confusion Matrix:
[[ 3625 12157]
 [ 1956 72179]]


In [17]:
import pandas as pd
import numpy as np
import pickle
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

# Définition des variables catégorielles et numériques
cat_features = ['State', 'UrbanRural', 'Bank', 'BankState', 'NewExist', 'FranchiseCode', 
                'IsFranchise', 'RevLineCr', 'LowDoc']
num_features = ['NoEmp', 'CreateJob', 'RetainedJob', 'Term', 'DisbursementGross', 'GrAppv']

# Séparation des features et de la target
X = df[cat_features + num_features]  
y = df['MIS_Status']

# Division en train/test **en conservant les noms de colonnes**
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, stratify=y)

# Transformer pour encodage et standardisation
preprocessor = ColumnTransformer([
    ("cat", OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1), cat_features),  # Encode les catégories
    ("num", StandardScaler(), num_features)  # Standardise les variables numériques
], remainder="passthrough")  # Les autres colonnes restent inchangées

# Création de la pipeline
pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier", LogisticRegression(max_iter=100, C=5.004577800250009, solver='lbfgs'))
])

# Entraînement
pipeline.fit(X_train, y_train)

/home/utilisateur/Documents/accord_pret/loan_project/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat',
                                                  OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  ['State', 'UrbanRural',
                                                   'Bank', 'BankState',
                                                   'NewExist', 'FranchiseCode',
                                                   'IsFranchise', 'RevLineCr',
                                                   'LowDoc']),
                                                 ('num', StandardScaler(),
                                                  ['NoEmp', 'CreateJob',
                                                   'RetainedJob', 'Term',
                                                   'DisbursementGross',
                                                   'GrAppv'])])),
                ('classifier', LogisticRegression(C=5.004577800250009))])